In [ ]:
#! pip install awswrangler

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import awswrangler

In [ ]:
session = awswrangler.Session()
df = session.pandas.read_sql_athena(
    sql="select * from resellers_sample",
    database="implementationdb"
)

In [ ]:
df_r = session.pandas.read_sql_athena(
    sql="select * from reseller",
    database="implementationdb"
)

In [ ]:
df.head()

In [ ]:
df_r.head()

In [ ]:
df.info()

In [ ]:
df_r.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

### Filter Last 4 Months

In [ ]:
max_date = df['date'].max()
min_date = max_date - pd.to_timedelta(120, unit='d')
df = df[df['date'] > min_date]

## Exploratory Data Analysis

In [ ]:
plt.hist(df[df["bill"] < df["bill"].quantile(0.95)]['bill']);

### Mean by day

Let's also analyse the mean by day to see if there's a trend or pattern in time

In [ ]:
df_mean_by_day = df.groupby('date')['bill'].mean()

In [ ]:
r = pd.date_range(start=min_date, end=max_date)
df_mean_by_day = df_mean_by_day.reindex(r).rename_axis('date').reset_index().fillna(0)

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.plot(df_mean_by_day['date'], df_mean_by_day['bill'])
plt.xticks(rotation='vertical');

In [ ]:
def completeItem(dfItem):
    min_date = dfItem['date'].min()
    max_date = dfItem['date'].max()
    if min_date == max_date:
        #only one data point
        return
    r = pd.date_range(start=min_date, end=max_date)
    dfItemNew = dfItem.set_index('date').reindex(r).rename_axis('date').reset_index()
    
    dfItemNew['mean-last-30'] = dfItemNew['bill'].rolling(30,min_periods=1).mean().reset_index()['bill']
    dfItemNew['mean-last-7'] = dfItemNew['bill'].rolling(7,min_periods=1).mean().reset_index()['bill']
    dfItemNew['std-last-30'] = dfItemNew['bill'].rolling(30,min_periods=1).std().reset_index()['bill']
    dfItemNew['bill'] = dfItemNew['bill'].fillna(0)
    dfItemNew['id_reseller'] = dfItem['id_reseller'].max()
    dfItemNew['std-last-30'].fillna(method='ffill',inplace=True)
    dfItemNew['mean-last-7'].fillna(method='ffill',inplace=True)
    dfItemNew['std-last-30'].fillna(method='ffill',inplace=True)
    resp = []
    counter = 0
    for index,row in dfItemNew.iterrows(): 
        resp.append(counter)
        if row['bill'] == 0: 
            counter += 1 
        else:
            counter = 0
    dfItemNew['days_without_purchase'] = pd.Series(resp)
    return dfItemNew

In [ ]:
i = 0
dfCompletedList = []
for nid,item in df.groupby('id_reseller'):
    i = i+1
    if i%200 == 0:
        print ('processed {} resellers'.format(str(i)))
    dfCompletedList.append(completeItem(item))

In [ ]:
df = pd.concat(dfCompletedList).copy()
del dfCompletedList
df['weekday']  = df['date'].dt.weekday_name


In [ ]:
df['next_bill'] = df.replace(0,np.nan).groupby('id_reseller')['bill'].fillna(method='bfill')

In [ ]:
df['last_bill'] = df.replace(0,np.nan).groupby('id_reseller')['bill'].fillna(method='ffill').copy()
different_zero = df['last_bill'].shift(1)
df.loc[df['bill'] != 0,'last_bill'] = np.nan
df['last_bill'] = df['last_bill'].fillna(different_zero)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = df.dropna().copy()

In [ ]:
#We meassure success computing how many resellers did we predict with less than 15% error.
def score_15(y,preds):
    return np.sum(np.abs((preds-y)/y) <= 0.15 )/ len(preds)

In [ ]:
score_15(df['next_bill'],df['last_bill'])

In [ ]:
df = pd.concat([df,pd.get_dummies(df.weekday)],axis=1)

In [ ]:
val_resellers = list(pd.Series(df['id_reseller'].unique()).sample(frac=0.1))
df_train = df[~df['id_reseller'].isin(val_resellers)].sample(frac=1)
df_validation = df[df['id_reseller'].isin(val_resellers)].sample(frac=1)

In [ ]:
df_train.head()

In [ ]:
df_train.columns

In [ ]:
X_train = df_train[['mean-last-30', 'mean-last-7',
       'std-last-30', 'days_without_purchase','last_bill', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday',
       'Tuesday', 'Wednesday']]

In [ ]:
y_train = df_train['next_bill']

In [ ]:
X_test = df_validation[['mean-last-30', 'mean-last-7',
       'std-last-30', 'days_without_purchase','last_bill', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday',
       'Tuesday', 'Wednesday']]

In [ ]:
y_test = df_validation['next_bill']

In [ ]:
model = RandomForestRegressor(n_estimators=40,n_jobs=-1)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
score_15(y_test,preds)

In [ ]:
objects = X_train.columns
y_pos = np.arange(len(objects))
performance = model.feature_importances_

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects);